<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Graph-Database-Error" data-toc-modified-id="Graph-Database-Error-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Graph Database Error</a></span><ul class="toc-item"><li><span><a href="#Package_name-error-in-package-vertex-instances-because-of-RPM-error" data-toc-modified-id="Package_name-error-in-package-vertex-instances-because-of-RPM-error-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Package_name error in package vertex instances because of RPM error</a></span></li><li><span><a href="#Number-of-total-vertex-instances-and-sum-of-total-vertex-instances-for-each-vertex-label-is-not-the-same" data-toc-modified-id="Number-of-total-vertex-instances-and-sum-of-total-vertex-instances-for-each-vertex-label-is-not-the-same-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Number of total vertex instances and sum of total vertex instances for each vertex label is not the same</a></span></li><li><span><a href="#Duplicates-of-packages-version-which-leads-to-more-has_artifact-edges-respect-to-the-number-of-artifacts" data-toc-modified-id="Duplicates-of-packages-version-which-leads-to-more-has_artifact-edges-respect-to-the-number-of-artifacts-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Duplicates of packages version which leads to more has_artifact edges respect to the number of artifacts</a></span></li><li><span><a href="#There-is-one-python-package-version-without-url_index-(maybe-it-is-also-the-missing-vertex-error-1.2)" data-toc-modified-id="There-is-one-python-package-version-without-url_index-(maybe-it-is-also-the-missing-vertex-error-1.2)-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>There is one python package version without url_index (maybe it is also the missing vertex error 1.2)</a></span></li><li><span><a href="#Why-RPM-packages-don't-have-url_index-yet?" data-toc-modified-id="Why-RPM-packages-don't-have-url_index-yet?-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Why RPM packages don't have url_index yet?</a></span></li></ul></li><li><span><a href="#Query-Optimization" data-toc-modified-id="Query-Optimization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Query Optimization</a></span><ul class="toc-item"><li><span><a href="#Analyze-get_error_python_packages_count(self,-unsolvable:-bool-=-False,-unparsable:-bool-=-False)" data-toc-modified-id="Analyze-get_error_python_packages_count(self,-unsolvable:-bool-=-False,-unparsable:-bool-=-False)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Analyze get_error_python_packages_count(self, unsolvable: bool = False, unparsable: bool = False)</a></span></li><li><span><a href="#Analyze-get_solver_documents_count(self)" data-toc-modified-id="Analyze-get_solver_documents_count(self)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Analyze get_solver_documents_count(self)</a></span></li><li><span><a href="#Analyze-get_analyzer_documents_count(self)" data-toc-modified-id="Analyze-get_analyzer_documents_count(self)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Analyze get_analyzer_documents_count(self)</a></span></li></ul></li></ul></div>

In [1]:
from thoth.storages.graph import GraphDatabase
from thoth.lab import GraphQueryResult as gqr

graph_db = GraphDatabase.create('janusgraph.test.thoth-station.ninja', port=8182)
graph_db.connect()
g = graph_db.g   # We will use raw Gremlin traversal in examples.

In [2]:
import pandas as pd
from pprint import pprint
from thoth.solver import pip_compile
from thoth.storages.graph.models import ALL_MODELS
from gremlin_python.process.graph_traversal import has
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import not_
from gremlin_python.process.traversal import P
from gremlin_python.process.graph_traversal import identity
from gremlin_python.process.graph_traversal import path
from gremlin_python.process.graph_traversal import outE
from gremlin_python.process.graph_traversal import out
from gremlin_python.process.graph_traversal import inE
from gremlin_python.process.graph_traversal import inV
from gremlin_python.process.graph_traversal import select
from gremlin_python.process.graph_traversal import values
from gremlin_python.process.graph_traversal import fold
from gremlin_python.process.graph_traversal import constant
from gremlin_python.process.graph_traversal import project

# Graph Database Error

## Package_name error in package vertex instances because of RPM error

In [ ]:
packages_vector_all = gqr(g.V().has('__label__', 'package').groupCount().by("package_name").next()).result

## Number of total vertex instances and sum of total vertex instances for each vertex label is not the same

In [ ]:
#Take ids of vertices instances for each label
vertices_ids_per_label = gqr(g.V().has("__label__").group().by("__label__").next()).result

In [ ]:
vertices_ids_total = gqr(g.V().has("__label__").valueMap(True).next()).result
print(vertices_ids_total)

In [ ]:
print(vertices_ids_per_label)

## Duplicates of packages version which leads to more has_artifact edges respect to the number of artifacts

## There is one python package version without url_index (maybe it is also the missing vertex error 1.2)

## Why RPM packages don't have url_index yet?

# Query Optimization

## Analyze get_error_python_packages_count(self, unsolvable: bool = False, unparsable: bool = False)

     def get_error_python_packages_count(self, unsolvable: bool = False, unparsable: bool = False) -> int:
        """Retrieve number of Python packages stored in the graph database with error flag."""
        if unsolvable and unparsable:
            raise ValueError("Properties unsolvable and unparsable are disjoin")

        query = (
            self.g.E()
            .has("__label__", Solved.__label__)
            .has("__type__", "edge")
            .has("solver_document_id")
            .has("solver_datetime")
            .has("solver_error", True)
            .has("solver_error_unsolvable", unsolvable)
            .has("solver_error_unparsable", unparsable)
            .inV()
            .has("__label__", PythonPackageVersion.__label__)
            .has("__type__", "vertex")
            .has("ecosystem", "pypi")
            .has("package_name")
            .has("package_version")
            .dedup()
            .count()
            .next()
        )

        return asyncio.get_event_loop().run_until_complete(query)

In [3]:
graph_db.get_error_python_packages_count(unsolvable=True)

3601

In [27]:
gqr(g.E()
    .has("__type__", "edge")
    .has("__label__", "solved")
    .has("solver_datetime")
    .has("solver_document_id")
    .has("solver_error", True)
    .has("solver_error_unsolvable", True)
    .inV()
    .has("__label__", "python_package_version")
    .has("__type__", "vertex")
    .has("ecosystem", "pypi")
    .has("package_name")
    .has("package_version")
    .dedup()
    .count()
    .next()
   ).result

3601

In [29]:
%%time
gqr(g.E()
    .has("__label__", "solved")
    .flatMap(
    has("__type__", "edge")
    .has("solver_datetime")
    .has("solver_document_id")
    .has("solver_error", True)
    .has("solver_error_unsolvable", True)
    .inV()
    .has("__label__", "python_package_version")
    .has("__type__", "vertex")
    .has("ecosystem", "pypi")
    .has("package_name")
    .has("package_version")
    )
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 3.08 ms, sys: 2.04 ms, total: 5.11 ms
Wall time: 17 s


3601

In [28]:
%%time
gqr(g.E()
    .has("__label__", "solved")
    .flatMap(
    has("__type__", "edge")
    )
    .has("solver_datetime")
    .has("solver_document_id")
    .has("solver_error", True)
    .has("solver_error_unsolvable", True)
    .inV()
    .has("__label__", "python_package_version")
    .has("__type__", "vertex")
    .has("ecosystem", "pypi")
    .has("package_name")
    .has("package_version")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 4.75 ms, sys: 995 µs, total: 5.75 ms
Wall time: 15.3 s


3601

In [13]:
%%time
gqr(g.E()
    .has("__label__", "solved")
    .has("solver_datetime")
    .has("solver_document_id")
    .has("solver_error", True)
    .has("solver_error_unsolvable", True)
    .inV()
    .has("__label__", "python_package_version")
    .has("__type__", "vertex")
    .has("ecosystem", "pypi")
    .has("package_name")
    .has("package_version")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 2.86 ms, sys: 3.54 ms, total: 6.4 ms
Wall time: 13 s


3601

## Analyze get_solver_documents_count(self)

    def get_solver_documents_count(self) -> int:
        """Get number of solver documents synced into graph."""
        query = (
            self.g.E()
            .has("__label__", Solved.__label__)
            .has("__type__", "edge")
            .has("solver_document_id")
            .has("solver_datetime")
            .has("solver_error")
            .has("solver_error_unsolvable")
            .has("solver_error_unparsable")
            .valueMap()
            .select("solver_document_id")
            .dedup()
            .count()
            .next()
        )

        return asyncio.get_event_loop().run_until_complete(query)

In [14]:
graph_db.get_solver_documents_count()

3068

In [58]:
profiling = gqr(g.E()
    .has("__label__", "solved")
    .has("__type__", "edge")
    .has("solver_datetime")
    .has("solver_document_id")
    .has("solver_error")
    .has("solver_error_unsolvable")
    .has("solver_error_unparsable")
    .valueMap()
    .select("solver_document_id")
    .dedup()
    .count()
    .profile()
    .next()
   ).result

print('TOTAL: ', profiling['@value']['dur'])
for metric in profiling['@value']['metrics']:
    print()
    print('dur', metric['@value']['dur'])
    print('counts', metric['@value']['counts'])
    print('name', metric['@value']['name'])
    print('annotations', metric['@value']['annotations'])
    print('id', metric['@value']['id'])

TOTAL:  97267.194222

dur 84653.575187
counts {'traverserCount': 405001, 'elementCount': 405001}
name JanusGraphStep([],[__label__.eq(solved), __type__.eq(edge)])
annotations {'percentDur': 87.0319904507464, 'condition': '(__label__ = solved AND __type__ = edge)', 'isFitted': 'true', 'query': 'multiKSQ[1]@2147483647', 'index': 'byEdge', 'orders': '[]', 'isOrdered': 'true'}
id 13.0.0()

dur 4739.137767
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_datetime],value), ProfileStep])
annotations {'percentDur': 4.872287933158143}
id 2.0.0()

dur 990.307358
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_document_id],value), ProfileStep])
annotations {'percentDur': 1.018130898008376}
id 3.0.0()

dur 909.773137
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_error],value), ProfileStep])
an

In [30]:
gqr(g.E()
    .has("__label__", "solved")
    .flatMap(
        has("__type__", "edge")
        .has("solver_document_id")
        .has("solver_datetime")
        .has("solver_error")
        .has("solver_error_unsolvable")
        .has("solver_error_unparsable")
    )
    .valueMap()
    .select("solver_document_id")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 4.12 ms, sys: 3.92 ms, total: 8.04 ms
Wall time: 22.2 s


3068

In [51]:
profiling = gqr(g.E()
    .has("__label__", "solved")
    .flatMap(
        has("__type__", "edge")
        .has("solver_datetime")
        .has("solver_document_id")
        .has("solver_error")
        .has("solver_error_unsolvable")
        .has("solver_error_unparsable")
    )
    .valueMap()
    .select("solver_document_id")
    .dedup()  
    .count()
    .profile()
    .next()
   ).result

print('TOTAL: ', profiling['@value']['dur'])
for metric in profiling['@value']['metrics']:
    print()
    print('dur', metric['@value']['dur'])
    print('counts', metric['@value']['counts'])
    print('name', metric['@value']['name'])
    print('annotations', metric['@value']['annotations'])
    print('id', metric['@value']['id'])

TOTAL:  25468.824184

dur 13914.399615
counts {'traverserCount': 405001, 'elementCount': 405001}
name JanusGraphStep([],[__label__.eq(solved)])
annotations {'percentDur': 54.63306634996244, 'condition': '(__label__ = solved)', 'isFitted': 'false', 'query': '[]', 'orders': '[]', 'isOrdered': 'true'}
id 9.0.0()

dur 7417.470736
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFlatMapStep([HasStep([__type__.eq(edge)]), ProfileStep, TraversalFilterStep([JanusGraphPropertiesStep([solver_datetime],value), ProfileStep]), ProfileStep, TraversalFilterStep([JanusGraphPropertiesStep([solver_document_id],value), ProfileStep]), ProfileStep, TraversalFilterStep([JanusGraphPropertiesStep([solver_error],value), ProfileStep]), ProfileStep, TraversalFilterStep([JanusGraphPropertiesStep([solver_error_unsolvable],value), ProfileStep]), ProfileStep, TraversalFilterStep([JanusGraphPropertiesStep([solver_error_unparsable],value), ProfileStep]), ProfileStep])
annotations {'percentDur': 

In [31]:
%%time
gqr(g.E()
    .has("__label__", "solved")
    .flatMap(
        has("__type__", "edge")
    )
    .has("solver_document_id")
    .has("solver_datetime")
    .has("solver_error")
    .has("solver_error_unsolvable")
    .has("solver_error_unparsable")
    .valueMap()
    .select("solver_document_id")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 6.42 ms, sys: 1.2 ms, total: 7.62 ms
Wall time: 19.6 s


3068

In [50]:
profiling = gqr(g.E()
    .has("__label__", "solved")
    .flatMap(
        has("__type__", "edge")
    )
    .has("solver_datetime")
    .has("solver_document_id")
    .has("solver_error")
    .has("solver_error_unsolvable")
    .has("solver_error_unparsable")
    .valueMap()
    .select("solver_document_id")
    .dedup()  
    .count()
    .profile()
    .next()
   ).result

print('TOTAL: ', profiling['@value']['dur'])
for metric in profiling['@value']['metrics']:
    print()
    print('dur', metric['@value']['dur'])
    print('counts', metric['@value']['counts'])
    print('name', metric['@value']['name'])
    print('annotations', metric['@value']['annotations'])
    print('id', metric['@value']['id'])

TOTAL:  22973.117253

dur 11340.427286
counts {'traverserCount': 405001, 'elementCount': 405001}
name JanusGraphStep([],[__label__.eq(solved)])
annotations {'percentDur': 49.36390286572486, 'condition': '(__label__ = solved)', 'isFitted': 'false', 'query': '[]', 'orders': '[]', 'isOrdered': 'true'}
id 14.0.0()

dur 2748.09499
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFlatMapStep([HasStep([__type__.eq(edge)]), ProfileStep])
annotations {'percentDur': 11.96222071099704}
id 2.0.0()

dur 957.057304
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_datetime],value), ProfileStep])
annotations {'percentDur': 4.165987982649678}
id 3.0.0()

dur 1595.575385
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_document_id],value), ProfileStep])
annotations {'percentDur': 6.945402173454009}
id 4.0.0()

dur 832.148331
counts {'traverserCoun

In [22]:
gqr(g.E()
    .has("__label__", "solved")
    .has("solver_document_id")
    .has("solver_datetime")
    .has("solver_error")
    .has("solver_error_unsolvable")
    .has("solver_error_unparsable")
    .valueMap()
    .select("solver_document_id")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 2.58 ms, sys: 3.68 ms, total: 6.26 ms
Wall time: 20.6 s


3068

In [49]:
profiling = gqr(g.E()
    .has("__label__", "solved")
    .has("solver_datetime")
    .has("solver_document_id")
    .has("solver_error")
    .has("solver_error_unsolvable")
    .has("solver_error_unparsable")
    .valueMap()
    .select("solver_document_id")
    .dedup()  
    .count()
    .profile()
    .next()
   ).result

print('TOTAL: ', profiling['@value']['dur'])
for metric in profiling['@value']['metrics']:
    print()
    print('dur', metric['@value']['dur'])
    print('counts', metric['@value']['counts'])
    print('name', metric['@value']['name'])
    print('annotations', metric['@value']['annotations'])
    print('id', metric['@value']['id'])

TOTAL:  23131.734001

dur 13313.258026
counts {'traverserCount': 405001, 'elementCount': 405001}
name JanusGraphStep([],[__label__.eq(solved)])
annotations {'percentDur': 57.55408576557408, 'condition': '(__label__ = solved)', 'isFitted': 'false', 'query': '[]', 'orders': '[]', 'isOrdered': 'true'}
id 13.0.0()

dur 1370.526515
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_datetime],value), ProfileStep])
annotations {'percentDur': 5.924875821850412}
id 2.0.0()

dur 905.446926
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_document_id],value), ProfileStep])
annotations {'percentDur': 3.914306320316742}
id 3.0.0()

dur 884.264159
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_error],value), ProfileStep])
annotations {'percentDur': 3.822731832217043}
id 4.0.0()

dur 887.002752
count

In [43]:
gqr(g.E()
    .has("solver_document_id")
    .valueMap()
    .select("solver_document_id")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 5.18 ms, sys: 1.53 ms, total: 6.71 ms
Wall time: 18.8 s


3068

In [48]:
profiling = gqr(g.E()
    .has("solver_document_id")
    .valueMap()
    .select("solver_document_id")
    .dedup()  
    .count()
    .profile()
    .next()
   ).result

print('TOTAL: ', profiling['@value']['dur'])
for metric in profiling['@value']['metrics']:
    print()
    print('dur', metric['@value']['dur'])
    print('counts', metric['@value']['counts'])
    print('name', metric['@value']['name'])
    print('annotations', metric['@value']['annotations'])
    print('id', metric['@value']['id'])

TOTAL:  19177.039377

dur 10238.090953
counts {'traverserCount': 637271, 'elementCount': 637271}
name JanusGraphStep(edge,[])
annotations {'percentDur': 53.38723434691939, 'condition': '()', 'isFitted': 'false', 'query': '[]', 'orders': '[]', 'isOrdered': 'true'}
id 8.0.0()

dur 5077.387064
counts {'traverserCount': 405001, 'elementCount': 405001}
name TraversalFilterStep([JanusGraphPropertiesStep([solver_document_id],value), ProfileStep])
annotations {'percentDur': 26.47638649628873}
id 1.0.0()

dur 3431.835047
counts {'traverserCount': 405001, 'elementCount': 405001}
name PropertyMapStep(value)
annotations {'percentDur': 17.89554153555097}
id 2.0.0()

dur 203.253568
counts {'traverserCount': 405001, 'elementCount': 405001}
name SelectOneStep(last,solver_document_id)
annotations {'percentDur': 1.059879807327153}
id 3.0.0()

dur 225.386191
counts {'traverserCount': 3068, 'elementCount': 3068}
name DedupGlobalStep
annotations {'percentDur': 1.175291902827905}
id 4.0.0()

dur 1.086554
co

## Analyze get_analyzer_documents_count(self)

    def get_analyzer_documents_count(self) -> int:
        """Get number of image analysis documents synced into graph."""
        query = (
            self.g.E()
            .has("__type__", "edge")
            .has("__label__", Requires.__label__)
            .has("analysis_datetime")
            .has("analysis_document_id")
            .has("analyzer_name")
            .has("analyzer_version")
            .valueMap()
            .select("analysis_document_id")
            .dedup()
            .count()
            .next()
        )

        return asyncio.get_event_loop().run_until_complete(query)

In [23]:
graph_db.get_analyzer_documents_count()

1

In [59]:
profiling = gqr(g.E()
    .has("__label__", "requires")
    .has("__type__", "edge")
    .has("analysis_datetime")
    .has("analysis_document_id")
    .has("analyzer_name")
    .has("analyzer_version")
    .valueMap()
    .select("analysis_document_id")
    .dedup()
    .count()
    .profile()
    .next()
   ).result

print('TOTAL: ', profiling['@value']['dur'])
for metric in profiling['@value']['metrics']:
    print()
    print('dur', metric['@value']['dur'])
    print('counts', metric['@value']['counts'])
    print('name', metric['@value']['name'])
    print('annotations', metric['@value']['annotations'])
    print('id', metric['@value']['id'])

TOTAL:  460.997366

dur 340.257041
counts {'traverserCount': 3350, 'elementCount': 3350}
name JanusGraphStep([],[__label__.eq(requires), __type__.eq(edge)])
annotations {'percentDur': 73.80889048290138, 'condition': '(__label__ = requires AND __type__ = edge)', 'isFitted': 'true', 'query': 'multiKSQ[1]@2147483647', 'index': 'byEdge', 'orders': '[]', 'isOrdered': 'true'}
id 12.0.0()

dur 49.844771
counts {'traverserCount': 3350, 'elementCount': 3350}
name TraversalFilterStep([JanusGraphPropertiesStep([analysis_datetime],value), ProfileStep])
annotations {'percentDur': 10.81237652884984}
id 2.0.0()

dur 13.11535
counts {'traverserCount': 3350, 'elementCount': 3350}
name TraversalFilterStep([JanusGraphPropertiesStep([analysis_document_id],value), ProfileStep])
annotations {'percentDur': 2.844994563374577}
id 3.0.0()

dur 11.182904
counts {'traverserCount': 3350, 'elementCount': 3350}
name TraversalFilterStep([JanusGraphPropertiesStep([analyzer_name],value), ProfileStep])
annotations {'per

In [24]:
%%time
gqr(g.E()
    .has("__label__", "requires")
    .flatMap(
    has("__type__", "edge")
    .has("analysis_datetime")
    .has("analysis_document_id")
    .has("analyzer_name")
    .has("analyzer_version")
    )
    .valueMap()
    .select("analysis_document_id")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 3.29 ms, sys: 1.27 ms, total: 4.57 ms
Wall time: 12.4 s


1

In [40]:
%%time
gqr(g.E()
    .has("__label__", "requires")
    .flatMap(
    has("__type__", "edge")
    )
    .has("analysis_datetime")
    .has("analysis_document_id")
    .has("analyzer_name")
    .has("analyzer_version")
    .valueMap()
    .select("analysis_document_id")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 5.28 ms, sys: 146 µs, total: 5.42 ms
Wall time: 10.9 s


1

In [26]:
%%time
gqr(g.E()
    .has("__label__", "requires")
    .has("analysis_datetime")
    .has("analysis_document_id")
    .has("analyzer_name")
    .has("analyzer_version")
    .valueMap()
    .select("analysis_document_id")
    .dedup()
    .count()
    .next()
   ).result

CPU times: user 3.46 ms, sys: 784 µs, total: 4.24 ms
Wall time: 11.1 s


1